When I first heard about data science, I did not really sure where should I start and what the path is to become a Data Scientist. After countless hours of research my conclusion about what the job consists of was unclear. What I had discovered in my research was how this job title is rapidly growing into a demanding career option. Living in a time of enormous amounts of data, more and more companies are in demand and looking to fill this position. I began to question, how do you become a data scientist, what degree or level of education do you need, what skills are employers looking for? This lead me to explore the job applications, backgrounds, experiences, and skills that the current data scientist possess. So the data analysis begins!
first, I scrape LinkedIn job and I supposed with this I would gather job descriptions and key skills demanding. Second I am going to scrape individuals who land the job as a Data Scientist.
For the first part I will go with getting insights from job descriptions. Let ‘s start scraping!


In [2]:
# selenium
from selenium.webdriver.common.by import By
import selenium.webdriver as webdriver
import selenium.webdriver.support.ui as ui
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#sys libraries
from time import sleep
import requests
import random
import csv
import io
import os
import sys
import types
import getpass
import string
import re
from bs4 import BeautifulSoup

# Import data viz libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# nltk and wordcloud
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud

%matplotlib inline

pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows' , 100)

### The Web-scrape:
I had to decide on what country and job title to pull job information from. I decided to go with United Kingdom due to the enormous amounts of applications for data scientists and the English language standard libraries for analyzing text data. 

>First, I do not condone scraping LinkedIn data in any way. Anyone who wishes to do so should first read <a href = 'https://www.linkedin.com/help/linkedin/answer/56347/prohibition-of-scraping-software?lang=enprohibition'>  LinkedIn's statement </a> on their of scraping software. 

>Also, note that LinkedIn actively suspends accounts due to <b>excessive activity</b> on a given account.

This code is to be used for learning purposes only and should not be used a tool for any commercial purpose. 

I used selenium and beautiful soup to web-scrape job application in United Kingdom in LinkedIn. 
When searching through the list of job application on LinkedIn, it will show a number of pages with 25 job applications on each page. After your first page, to continue scraping on to the next set of 25 job applications you have to get to the next page. Using selenium helped me maneuver around this issue. Selenium has a restriction on its speed because the scraping with the browser is much slower. Due to the slowness, to not get banned by LinkedIn the use of the "sleep" statements had to be used in my code multiple times to cause further slow-down. I write every job applicant to a csv file.





#### linkedin account config

In [2]:
# getting from user input
# try:
#     username = input('Enter your Username ')
#     password = getpass.getpass('Enter your password ')
#     desired_job = input('Searched job query: ')
#     desired_location = input('Location: ')
# except Exception as error: 
#     print('ERROR', error) 
    
    
username = 'behdad.k7@gmail.com'
password = '1988behrooz10973'
desired_keyword = 'data scientist in united kingdom'
desired_location = 'United Kingdom'

csv_file = str.strip(desired_keyword)
file_name = csv_file + "_" + str(desired_location) +'.csv'

In [22]:
df_1 = pd.read_csv('result.csv',index_col=False)
link_df = df_1[df_1['present_url'].notnull()]
link_df

,name,job_title,current_company,university,degree,location,about,skills,present_url
0,Andria Sarri,Data Scientist,HSBC,UCL,Doctor of Philosophy (Ph.D.),"London, Greater London, United Kingdom",NaN,Data Analysis,https://www.linkedin.com/in/andriasarri/
1,Saksham Jindal,Data Scientist at Fractal Analytics,Fractal Analytics,"Indian Institute of Technology, Kharagpur",Dual Degree (Bachelor of Technology (B.Tech.) + Masters of Technology (M.Tech.)),"London, United Kingdom",NaN,C++,https://www.linkedin.com/in/saksham-jindal-52b7b47a/
2,sasan raeissian,Senior Data Scientist at LGS - An IBM Company,LGS - An IBM Company,Université de Montréal,Master of Science (MSc),"Montreal, Canada Area",Summary of qualifications\r\n\r\nProfessionally qualified statistician with strong programming b...,NaN,https://www.linkedin.com/in/sasan-raeissian-b4b1a034/
3,Rahul Jaiswal,Data Scientist at HCL Technologies,HCL Technologies,Academy of Technology,BTech - Bachelor of Technology,"London, United Kingdom","Machine learning,Deep learning, Python,R ,NLP(Spacy,nltk,gensim), Statistical analysis.I have 3+...",NaN,https://www.linkedin.com/in/rahul-jaiswal-570042ab/
4,Subbarao Y,Artificial Intelligence machine learning - Chatbot developer,DBS Bank,NaN,NaN,"Nellore, Andhra Pradesh, India",*IT Professional with 11 Years of Experience in Software industry.\r\n*good working knowledge on...,NaN,https://www.linkedin.com/in/subbarao-y-1547a8134/
...,...,...,...,...,...,...,...,...,...
661,Mahsa Daei,Human Resources Manager at Ariana Labs (Hiring Now!),Ariana Labs,NaN,NaN,Iran,NaN,NaN,https://www.linkedin.com/in/mahsadaei/
662,Elham Joodi,Senior HR Specialist at Quiz Of Kings,Quiz of Kings,NaN,NaN,Iran,NaN,Recruiting,https://www.linkedin.com/in/elham-joodi-8b41a9147/
663,Baran Shojaee,Human Resources Manager at Kian Capital,Kian Capital,NaN,NaN,"London, Greater London, United Kingdom",NaN,NaN,https://www.linkedin.com/in/baranshojaee/
664,Dr. Farzaneh Soltani (PHD),vice president of CEO at BUFFALO -Married,Allameh Tabataba'i University,NaN,NaN,Iran,I live in Tehran. Experienced Administrative Consultant with a demonstrated history of working i...,NaN,https://www.linkedin.com/in/dr-farzaneh-soltani-phd-abbb3059/


In [19]:
p_list = link_df['present_url'].tolist()
print(len(p_list))

666


#### Scraping Linkedin jobs and save in a CSV file

In [20]:
driver = webdriver.Firefox(executable_path= "geckodriver.exe")

profile = webdriver.FirefoxProfile()
profile.set_preference("browser.cache.disk.enable", False)
profile.set_preference("browser.cache.memory.enable", False)
profile.set_preference("browser.cache.offline.enable", False)
profile.set_preference("network.http.use-cache", False)
profile.update_preferences()

driver.get(url='https://www.linkedin.com/')
sleep(2)

# Find the sign in botthon
sign_in = driver.find_element(by='link text', value='Sign in')
sign_in.click()
sleep(5)

# We find the input section and enter our user information.
email_entry = driver.find_element(by='css selector', value='#username')
password_entry = driver.find_element(by='css selector', value='#password')

# For every letter in our email, we will set a random time between keystrokes
for letter in username:
    sleep(random.uniform(.1, .4))
    email_entry.send_keys(letter)

# We expect a random time between 0.1 and 0.4 sec for each letter of our password.
for letter in password:
    sleep(random.uniform(.1, .4))
    password_entry.send_keys(letter)

# We press enter
password_entry.send_keys(Keys.RETURN)
sleep(10)

# click on typehead search bar
search_bar = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "//input[@placeholder='Search']")))
search_bar.click()

# Enter the job search term and press enter
for letter in desired_keyword:
    sleep(random.uniform(.1, .4))
    search_bar.send_keys(letter)
search_bar.send_keys(Keys.RETURN)
sleep(3)


# Find and Click on just people button
just_people_btn = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//button[@aria-label='View only People results']")))
just_people_btn.click()
sleep(2)

source = requests.get(driver.current_url).text
soup = BeautifulSoup(source, 'lxml')

# open the csv file to wirte
writer = csv.writer(open(file_name, 'w', encoding='utf-8'))
#write the headers
writer.writerow(['name', 'job_title', 'current_company', 'university','degree', 'location', 'about', 'skills'
                        ,'present_url'])

for pro in p_list:
    try:
        driver.get(pro)
        sleep(10)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(1)
    except:
        pass
    
    links = driver.find_elements_by_xpath('//section[contains(@class, "pv-profile-section pv-browsemap-section profile-section artdeco-container-card ember-view")]//a[@href]')
    profiles_link = set()
    for link in links:
        link_add=link.get_attribute('href')
        print(link_add)
        if link_add.startswith('https://www.linkedin.com/in/'):
            profiles_link.add(link_add)
            
    for profile in profiles_link:
        try:
            driver.get(profile)
            sleep(5)
        except:
            pass
            

        # -------------Scrap here----------------
            
        print('---------------------------')
        print(profile)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        sleep(.5)
        

        ''' 1)Name'''
        try:
            name = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//li[contains(@class, "inline t-24 t-black t-normal break-words")]')))
            name = name.text
            if name:
                name = name.strip()
        except:
            name = np.nan
                
        print('name:\n',name , '\n')
        sleep(0.5)
        
        ''' 2)job_title'''
        try:
            job_title = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//h2[contains(@class, "mt1 t-18 t-black t-normal")]')))
            job_title = job_title.text
            if job_title:
                job_title = job_title.strip()
        except:
            job_title = np.nan
                
        print('job_title:\n',job_title , '\n')
        sleep(0.5)
        
        ''' 3)current_company'''
        try:
            current_company = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//ul[contains(@class, "pv-top-card--experience-list")]/li[1]')))
            current_company = current_company.text
            if current_company:
                current_company = current_company.strip()
        except:
            current_company = np.nan
                
        print('current_company:\n',current_company , '\n')
        sleep(0.5)
        
        ''' 4)university'''
        try:
            university = WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((
                    By.XPATH,'//ul[contains(@class, "pv-top-card--experience-list")]/li[2]')))
            if university =='':
                degree = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((
                        By.XPATH,'//section[contains(@class, "pv-profile-section education-section ember-view")]')))
            degree = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//li[1]//div[contains(@class,"pv-entity__degree-info")]')))
            degree = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//h3[contains(@class,"pv-entity__school-name t-16 t-black t-bold")]')))
            
            university = university.text
            if university:
                university = university.strip()
        except:
            university = np.nan
                

        print('university:\n',university , '\n')
        sleep(0.5)
        
        
        ''' 5)degree'''
        try:
            degree = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//section[contains(@class, "pv-profile-section education-section ember-view")]')))
            degree = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//li[1]//div[contains(@class,"pv-entity__degree-info")]')))
            degree = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//span[contains(@class,"pv-entity__comma-item")]')))
            degree = degree.text
            if degree:
                degree = degree.strip()
        except:
            degree = np.nan
                
        print('degree:\n',degree , '\n')
        sleep(0.5)
        
        ''' 6)location'''
        try:
            location = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((
                    By.XPATH,'//ul[contains(@class, "pv-top-card--list pv-top-card--list-bullet mt1")]/li[1]')))
            location = location.text
            if location:
                location = location.strip()
        except:
            location = np.nan
        
        print('location:\n',location , '\n')
        sleep(0.5)
        
        ''' 7)about'''
        try:
            about = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((
                    By.XPATH,'//section[contains(@class, "artdeco-container-card pv-profile-section pv-about-section ember-view")]//a[contains(@class,"lt-line-clamp__more")]'))).click()
            about = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((
                    By.XPATH,'//p[contains(@class,"pv-about__summary-text mt4 t-14 ember-view")]')))      
            about = about.text
            if about:
                about = about.strip()
        except:
            about = np.nan
             
        print('about:\n',about , '\n')
        sleep(0.5)
        
        ''' 8)skills'''
        
        try:
            print("skills search ... ")
            skills = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((
                    By.XPATH,'//li[contains(@class , "pv-skill-category-entity__top-skill pv-skill-category-entity pb3 pt4 pv-skill-endorsedSkill-entity relative ember-view")]//span[contains(@class, "pv-skill-category-entity__name-text t-16 t-black t-bold")]')))
            skills = skills.text
            if skills:
                skills = skills.strip()
        
        except:
            skills = np.nan
                
        print('skills:\n',skills , '\n')
        sleep(0.5)
        
        
        
        writer.writerow([ name, job_title, current_company, university, degree, location, about
                        ,skills, profile])
            
        sleep(2)

print("The process is completed .../n closing The program!")
driver.quit()


https://www.linkedin.com/in/tsatska/
https://www.linkedin.com/in/olgapolchlopek/
https://www.linkedin.com/in/darragh-kelly-b952a555/
https://www.linkedin.com/in/sarah-miller-7026b8121/
https://www.linkedin.com/in/aycakandur/
https://www.linkedin.com/in/artemisnika/
https://www.linkedin.com/in/yu-lan-scholliers-70138710/
https://www.linkedin.com/in/aidanrussell/
https://www.linkedin.com/in/mehtab-pathan-phd-62950066/
https://www.linkedin.com/in/ana-camila-baez-0071129b/
---------------------------
https://www.linkedin.com/in/aidanrussell/
name:
 Aidan Russell 

job_title:
 Freelance Data Scientist 

current_company:
 Rolls-Royce 

university:
 nan 

degree:
 nan 

location:
 London, United Kingdom 

about:
 Freelance senior data scientist generally based in London and available for work across Europe. I have a strong interest in mathematics and its application in machine learning. Experienced on a range of projects from legacy codebases in large organisations through to building a start

skills search ... 
skills:
 nan 

---------------------------
https://www.linkedin.com/in/sachinyadav00/
name:
 Sachin Yadav 

job_title:
 Data Scientist at Fractal Analytics 

current_company:
 Fractal Analytics 

university:
 nan 

degree:
 Master’s Degree 

location:
 Bengaluru, Karnataka, India 

about:
 nan 

skills search ... 
skills:
 Machine Learning 

---------------------------
https://www.linkedin.com/in/aayush-rampal-36260574/
name:
 Aayush Rampal 

job_title:
 Data Scientist at Fractal Analytics 

current_company:
 Fractal Analytics 

university:
 nan 

degree:
 nan 

location:
 East Delhi, Delhi, India 

about:
 nan 

skills search ... 
skills:
 nan 

---------------------------
https://www.linkedin.com/in/swati-shaw-2298a8162/
name:
 Swati Shaw 

job_title:
 Data Scientist at Flipkart 

current_company:
 Flipkart 

university:
 Indian Institute of Technology, Delhi 

degree:
 Bachelor of Engineering - BE 

location:
 Bengaluru, Karnataka, India 

about:
 nan 

skills sea

skills search ... 
skills:
 nan 

---------------------------
https://www.linkedin.com/in/ranicolas/
name:
 Ramzy N. 

job_title:
 Data Scientist at FreshBooks 

current_company:
 FreshBooks 

university:
 University of Toronto 

degree:
 Bachelor of Science (B.Sc.) 

location:
 Toronto, Ontario, Canada 

about:
 nan 

skills search ... 
skills:
 nan 

---------------------------
https://www.linkedin.com/in/soniaabdoli/
name:
 Sonia Abdoli 

job_title:
 Deep Learning Engineer || Data Scientist 

current_company:
 SelfEmotion 

university:
 Université de Montréal 

degree:
 Master of Science - MS 

location:
 Montreal, Quebec, Canada 

about:
 • I am a Deep Learning Engineer & Data Scientist || My passion is using technology to solve the real world problem.
• I code to build tools, which help up to solve the problem. I 'm very interested in solving complex problems in their spare time.
• Research and Development Experiences.
• Consistently commended for programming abilities, strong gra

skills search ... 
skills:
 nan 

---------------------------
https://www.linkedin.com/in/mariosmichailidis/
name:
 Marios Michailidis 

job_title:
 Data Scientist 

current_company:
 H2O.ai 

university:
 nan 

degree:
 nan 

location:
 United Kingdom 

about:
 Experienced analyst that has worked both in Credit and Marketing sectors of the UK market. Have led analytics' projects with various themes including Acquisition, Retention (Loyalty), Uplift , fraud detection, portfolio optimization and more.

I am also the Creator and administrator of KazAnova , project made in Java for quick credit scoring and analytics (link: http://www.kazanovaforanalytics.com/).

I also enjoy participating in predictive modelling competitions, where I have won multiple times.

Specifically I like competing on Kaggle.com (a popular platform that holds predictive modelling competitions) , In February 2016 I held:

World Rank: 1st

out of 465,000 data scientists

http://blog.kaggle.com/2016/02/10/profiling-to

skills search ... 
skills:
 nan 

---------------------------
https://www.linkedin.com/in/kedarjoshi5231/
name:
 Kedar Joshi 

job_title:
 Data Science | Machine Learning | Business Intelligence 

current_company:
 Board Infinity 

university:
 nan 

degree:
 nan 

location:
 Pune, Maharashtra, India 

about:
 *Actively seeking a job in the industry.*

I am a huge data science enthusiast. I am an "Experienced fresher" looking for a full-time opportunity in the field of Data Science, Analytics, Machine Learning and Business Intelligence. I have completed my bachelor’s degree in Computer Science & Engineering from Visvesvaraya Technological University. I have completed my higher education from SGM College, Karad. After completion of my engineering, I was looking for a job, meanwhile I was searching for new courses as well which could be very useful to build my career. Board Infinity’s Data Science Learning Path is one of them. Coming to my technical skills I have strong knowledge of Adva

...

skills search ... 
skills:
 nan 

---------------------------
https://www.linkedin.com/in/samira-heidari-26719968/
name:
 Samira Heidari 

job_title:
 Rock Mechanics Expert, Freelancer at Pöyry Energy Ltd, Looking for new opportunities inside United States 

current_company:
 Pöyry Energy Ltd 

university:
 University of Tehran 

degree:
 Master’s Degree 

location:
 Iran 

about:
 Strong industry knowledge, comprehensive hands-on experience in rock mechanics works, Speaking fluent English and native-level Farsi.

- Experienced in design and numerical analysis of Underground structures (tunnels, caverns, e.tc.), Dams and Slopes.
- Solid knowledge of tunnelling methods and equipment, dam construction procedure.
- Solid knowledge of PLAXIS, Rocscience Software package, ITASCA Codes, Microsoft Office environment, and AUTOCAD software.
- Acquainted with interpreting Dam instrumentation and monitoring results.
- Acquainted with field investigation and geological mapping.
- Acquainted with p

location:
 United Kingdom 

about:
 nan 

skills search ... 
skills:
  

---------------------------
https://www.linkedin.com/in/zoe-dennis-52b6442b/
name:
 Zoe Dennis 

job_title:
 Land Manager at Linden Homes 

current_company:
 Linden Homes 

university:
 University of Kent 

degree:
 Bachelor’s Degree 

location:
 London, United Kingdom 

about:
 nan 

skills search ... 
skills:
  

---------------------------
https://www.linkedin.com/in/bwsutton1990/
name:
 Benjamin Sutton 

job_title:
 Data Scientist at Brit Insurance 

current_company:
 Brit Insurance 

university:
 The Open University 

degree:
 Bachelor of Science - BS 

location:
 London, Greater London, United Kingdom 

about:
 I am an analytical thinker who is driven to excel in everything I take on. I thrive in high pressure environments and am always competing to get the most challenging pieces of work to prove this time and time again.

I am eager to develop and am always looking to learn new skills, whilst simultaneousl

---------------------------
https://www.linkedin.com/in/priyadarshini-lakshminarasimhan-a0756228/
name:
 Priyadarshini Lakshminarasimhan 

job_title:
 Agile Tester at ZappiStore 

current_company:
 ZappiStore 

university:
 East Stroudsburg University of Pennsylvania 

degree:
 Master's Degree 

location:
 London, United Kingdom 

about:
 nan 

skills search ... 
skills:
  

---------------------------
https://www.linkedin.com/in/stephenhambling/
name:
 Stephen Hambling 

job_title:
 Cloud planning Expert, CEO of largest European Adaptive Insights Partner 

current_company:
 Formulate - A Kainos company 

university:
 nan 

degree:
 nan 

location:
 Madresfield, Hereford and Worcester, United Kingdom 

about:
 I have been in finance all my working life. From qualifying, in practice with Ernst and Young, managing a team to becoming a finance system specialist in SAP. Now I I head up Formulate, the largest European Adaptive Insights Partner; we build cloud planning solutions in Adaptive 

name:
 nan 

job_title:
 nan 

current_company:
 nan 

university:
 nan 

degree:
 nan 

location:
 nan 

about:
 nan 

skills search ... 
skills:
 nan 

---------------------------
https://www.linkedin.com/in/gaurav14cs17/
name:
 nan 

job_title:
 nan 

current_company:
 nan 

university:
 nan 

degree:
 nan 

location:
 nan 

about:
 nan 

skills search ... 
skills:
 nan 

---------------------------
https://www.linkedin.com/in/sathyanarayanan-o/
name:
 nan 

job_title:
 nan 

current_company:
 nan 

university:
 nan 

degree:
 nan 

location:
 nan 

about:
 nan 

skills search ... 
skills:
 nan 

---------------------------
https://www.linkedin.com/in/krishnaprasad-r-78187912a/
name:
 nan 

job_title:
 nan 

current_company:
 nan 

university:
 nan 

degree:
 nan 

location:
 nan 

about:
 nan 

skills search ... 
skills:
 nan 



InvalidSessionIdException: Message: Tried to run command without establishing a connection


## Merging

In [23]:
df_2 = pd.read_csv('data scientist in united kingdom_United Kingdom.csv',index_col=False)
df_2


,name,job_title,current_company,university,degree,location,about,skills,present_url
0,Aidan Russell,Freelance Data Scientist,Rolls-Royce,NaN,NaN,"London, United Kingdom",Freelance senior data scientist generally based in London and available for work across Europe. ...,NaN,https://www.linkedin.com/in/aidanrussell/
1,"Mehtab Pathan, PhD",Senior Data Scientist at Builder,Builder,Imperial College London,Doctor of Philosophy (PhD),"London, Greater London, United Kingdom",NaN,NaN,https://www.linkedin.com/in/mehtab-pathan-phd-62950066/
2,Sarah Miller,Data Scientist at KPMG,KPMG,"Baruch College, City University of New York (CUNY)",Doctor of Philosophy (Ph.D.),"London, Greater London, United Kingdom",New American implant in London!\r\n\r\nAn established and professional Analyst and Data Scientis...,Python,https://www.linkedin.com/in/sarah-miller-7026b8121/
3,Olga Półchłopek,Data Scientist at BCG GAMMA,BCG GAMMA,Vrije Universiteit Amsterdam (VU Amsterdam),Master's degree,"London, Greater London, United Kingdom",NaN,NaN,https://www.linkedin.com/in/olgapolchlopek/
4,Tsatska Enkhbayar,Data Scientist at Facebook,Facebook,Imperial College London,Bachelor's Degree,United Kingdom,NaN,Data Analysis,https://www.linkedin.com/in/tsatska/
...,...,...,...,...,...,...,...,...,...
418,Mohit Sonania,Data Scientist at BunkerEx,BunkerEx,Queen Mary University of London,Master's degree,"London, Greater London, United Kingdom",NaN,NaN,https://www.linkedin.com/in/mohitsonania/
419,Nadbor Drozd,Lead Data Scientist at Lloyds Banking Group,Lloyds Banking Group,University of Warsaw,M.S,"London, Greater London, United Kingdom",I love solving puzzles. I don't like to babble.\r\n\r\nIf you're a recruiter: http://nadbordrozd...,NaN,https://www.linkedin.com/in/nadbor-drozd-12316063/
420,Camille Reyes,Data Scientist,BBOXX,NaN,Bachelor's degree,"London, Greater London, United Kingdom",NaN,NaN,https://www.linkedin.com/in/camille-reyes/
421,Christopher Ottesen,Data Scientist,BP,NaN,NaN,"London, Greater London, United Kingdom",NaN,NaN,https://www.linkedin.com/in/christopher-ottesen-4bb10b51/


In [24]:
frames =[df_1, df_2]
result = pd.concat(frames)
result = result[result['name'].notnull()]
result

,name,job_title,current_company,university,degree,location,about,skills,present_url
0,Andria Sarri,Data Scientist,HSBC,UCL,Doctor of Philosophy (Ph.D.),"London, Greater London, United Kingdom",NaN,Data Analysis,https://www.linkedin.com/in/andriasarri/
1,Saksham Jindal,Data Scientist at Fractal Analytics,Fractal Analytics,"Indian Institute of Technology, Kharagpur",Dual Degree (Bachelor of Technology (B.Tech.) + Masters of Technology (M.Tech.)),"London, United Kingdom",NaN,C++,https://www.linkedin.com/in/saksham-jindal-52b7b47a/
2,sasan raeissian,Senior Data Scientist at LGS - An IBM Company,LGS - An IBM Company,Université de Montréal,Master of Science (MSc),"Montreal, Canada Area",Summary of qualifications\r\n\r\nProfessionally qualified statistician with strong programming b...,NaN,https://www.linkedin.com/in/sasan-raeissian-b4b1a034/
3,Rahul Jaiswal,Data Scientist at HCL Technologies,HCL Technologies,Academy of Technology,BTech - Bachelor of Technology,"London, United Kingdom","Machine learning,Deep learning, Python,R ,NLP(Spacy,nltk,gensim), Statistical analysis.I have 3+...",NaN,https://www.linkedin.com/in/rahul-jaiswal-570042ab/
4,Subbarao Y,Artificial Intelligence machine learning - Chatbot developer,DBS Bank,NaN,NaN,"Nellore, Andhra Pradesh, India",*IT Professional with 11 Years of Experience in Software industry.\r\n*good working knowledge on...,NaN,https://www.linkedin.com/in/subbarao-y-1547a8134/
...,...,...,...,...,...,...,...,...,...
418,Mohit Sonania,Data Scientist at BunkerEx,BunkerEx,Queen Mary University of London,Master's degree,"London, Greater London, United Kingdom",NaN,NaN,https://www.linkedin.com/in/mohitsonania/
419,Nadbor Drozd,Lead Data Scientist at Lloyds Banking Group,Lloyds Banking Group,University of Warsaw,M.S,"London, Greater London, United Kingdom",I love solving puzzles. I don't like to babble.\r\n\r\nIf you're a recruiter: http://nadbordrozd...,NaN,https://www.linkedin.com/in/nadbor-drozd-12316063/
420,Camille Reyes,Data Scientist,BBOXX,NaN,Bachelor's degree,"London, Greater London, United Kingdom",NaN,NaN,https://www.linkedin.com/in/camille-reyes/
421,Christopher Ottesen,Data Scientist,BP,NaN,NaN,"London, Greater London, United Kingdom",NaN,NaN,https://www.linkedin.com/in/christopher-ottesen-4bb10b51/


In [25]:
result.to_csv("result.csv", header=True, index = False)

In [3]:
df = pd.read_csv('result.csv',index_col=False)
df

,name,job_title,current_company,university,degree,location,about,skills,present_url
0,Andria Sarri,Data Scientist,HSBC,UCL,Doctor of Philosophy (Ph.D.),"London, Greater London, United Kingdom",NaN,Data Analysis,https://www.linkedin.com/in/andriasarri/
1,Saksham Jindal,Data Scientist at Fractal Analytics,Fractal Analytics,"Indian Institute of Technology, Kharagpur",Dual Degree (Bachelor of Technology (B.Tech.) + Masters of Technology (M.Tech.)),"London, United Kingdom",NaN,C++,https://www.linkedin.com/in/saksham-jindal-52b7b47a/
2,sasan raeissian,Senior Data Scientist at LGS - An IBM Company,LGS - An IBM Company,Université de Montréal,Master of Science (MSc),"Montreal, Canada Area",Summary of qualifications\r\n\r\nProfessionally qualified statistician with strong programming b...,NaN,https://www.linkedin.com/in/sasan-raeissian-b4b1a034/
3,Rahul Jaiswal,Data Scientist at HCL Technologies,HCL Technologies,Academy of Technology,BTech - Bachelor of Technology,"London, United Kingdom","Machine learning,Deep learning, Python,R ,NLP(Spacy,nltk,gensim), Statistical analysis.I have 3+...",NaN,https://www.linkedin.com/in/rahul-jaiswal-570042ab/
4,Subbarao Y,Artificial Intelligence machine learning - Chatbot developer,DBS Bank,NaN,NaN,"Nellore, Andhra Pradesh, India",*IT Professional with 11 Years of Experience in Software industry.\r\n*good working knowledge on...,NaN,https://www.linkedin.com/in/subbarao-y-1547a8134/
...,...,...,...,...,...,...,...,...,...
1084,Mohit Sonania,Data Scientist at BunkerEx,BunkerEx,Queen Mary University of London,Master's degree,"London, Greater London, United Kingdom",NaN,NaN,https://www.linkedin.com/in/mohitsonania/
1085,Nadbor Drozd,Lead Data Scientist at Lloyds Banking Group,Lloyds Banking Group,University of Warsaw,M.S,"London, Greater London, United Kingdom",I love solving puzzles. I don't like to babble.\r\n\r\nIf you're a recruiter: http://nadbordrozd...,NaN,https://www.linkedin.com/in/nadbor-drozd-12316063/
1086,Camille Reyes,Data Scientist,BBOXX,NaN,Bachelor's degree,"London, Greater London, United Kingdom",NaN,NaN,https://www.linkedin.com/in/camille-reyes/
1087,Christopher Ottesen,Data Scientist,BP,NaN,NaN,"London, Greater London, United Kingdom",NaN,NaN,https://www.linkedin.com/in/christopher-ottesen-4bb10b51/


In [4]:
print("Number of records: " , df.shape[0])

Number of records:  1089


In [5]:
df.drop_duplicates(inplace=True)
df

,name,job_title,current_company,university,degree,location,about,skills,present_url
0,Andria Sarri,Data Scientist,HSBC,UCL,Doctor of Philosophy (Ph.D.),"London, Greater London, United Kingdom",NaN,Data Analysis,https://www.linkedin.com/in/andriasarri/
1,Saksham Jindal,Data Scientist at Fractal Analytics,Fractal Analytics,"Indian Institute of Technology, Kharagpur",Dual Degree (Bachelor of Technology (B.Tech.) + Masters of Technology (M.Tech.)),"London, United Kingdom",NaN,C++,https://www.linkedin.com/in/saksham-jindal-52b7b47a/
2,sasan raeissian,Senior Data Scientist at LGS - An IBM Company,LGS - An IBM Company,Université de Montréal,Master of Science (MSc),"Montreal, Canada Area",Summary of qualifications\r\n\r\nProfessionally qualified statistician with strong programming b...,NaN,https://www.linkedin.com/in/sasan-raeissian-b4b1a034/
3,Rahul Jaiswal,Data Scientist at HCL Technologies,HCL Technologies,Academy of Technology,BTech - Bachelor of Technology,"London, United Kingdom","Machine learning,Deep learning, Python,R ,NLP(Spacy,nltk,gensim), Statistical analysis.I have 3+...",NaN,https://www.linkedin.com/in/rahul-jaiswal-570042ab/
4,Subbarao Y,Artificial Intelligence machine learning - Chatbot developer,DBS Bank,NaN,NaN,"Nellore, Andhra Pradesh, India",*IT Professional with 11 Years of Experience in Software industry.\r\n*good working knowledge on...,NaN,https://www.linkedin.com/in/subbarao-y-1547a8134/
...,...,...,...,...,...,...,...,...,...
1082,Anna Padkavyrava,Data Scientist at Brave Bison,Brave Bison,Belarusian State University of Informatics and Radioelectronics,Master’s Degree,"London, Greater London, United Kingdom",NaN,NaN,https://www.linkedin.com/in/apadkavyrava/
1083,Konstantinos Bozas,Senior Data Scientist at Amazon | Machine Learning,Amazon,"Queen Mary, U. of London",PhD,"London, United Kingdom",A goal oriented data scientist with research experience in machine learning and computer vision....,NaN,https://www.linkedin.com/in/kbozas/
1085,Nadbor Drozd,Lead Data Scientist at Lloyds Banking Group,Lloyds Banking Group,University of Warsaw,M.S,"London, Greater London, United Kingdom",I love solving puzzles. I don't like to babble.\r\n\r\nIf you're a recruiter: http://nadbordrozd...,NaN,https://www.linkedin.com/in/nadbor-drozd-12316063/
1087,Christopher Ottesen,Data Scientist,BP,NaN,NaN,"London, Greater London, United Kingdom",NaN,NaN,https://www.linkedin.com/in/christopher-ottesen-4bb10b51/


In [6]:
print("Number of records after removing duplicates: " , df.shape[0])

Number of records after removing duplicates:  765


In [11]:
df.to_csv("result_unique.csv", header=True, index = False)

## Exploratory Data Analysis (EDA)

In [12]:
# Reading from scv file
df = pd.read_csv('result_unique.csv',index_col=False)
df

,name,job_title,current_company,university,degree,location,about,skills,present_url
0,Andria Sarri,Data Scientist,HSBC,UCL,Doctor of Philosophy (Ph.D.),"London, Greater London, United Kingdom",NaN,Data Analysis,https://www.linkedin.com/in/andriasarri/
1,Saksham Jindal,Data Scientist at Fractal Analytics,Fractal Analytics,"Indian Institute of Technology, Kharagpur",Dual Degree (Bachelor of Technology (B.Tech.) + Masters of Technology (M.Tech.)),"London, United Kingdom",NaN,C++,https://www.linkedin.com/in/saksham-jindal-52b7b47a/
2,sasan raeissian,Senior Data Scientist at LGS - An IBM Company,LGS - An IBM Company,Université de Montréal,Master of Science (MSc),"Montreal, Canada Area",Summary of qualifications\r\n\r\nProfessionally qualified statistician with strong programming b...,NaN,https://www.linkedin.com/in/sasan-raeissian-b4b1a034/
3,Rahul Jaiswal,Data Scientist at HCL Technologies,HCL Technologies,Academy of Technology,BTech - Bachelor of Technology,"London, United Kingdom","Machine learning,Deep learning, Python,R ,NLP(Spacy,nltk,gensim), Statistical analysis.I have 3+...",NaN,https://www.linkedin.com/in/rahul-jaiswal-570042ab/
4,Subbarao Y,Artificial Intelligence machine learning - Chatbot developer,DBS Bank,NaN,NaN,"Nellore, Andhra Pradesh, India",*IT Professional with 11 Years of Experience in Software industry.\r\n*good working knowledge on...,NaN,https://www.linkedin.com/in/subbarao-y-1547a8134/
...,...,...,...,...,...,...,...,...,...
760,Anna Padkavyrava,Data Scientist at Brave Bison,Brave Bison,Belarusian State University of Informatics and Radioelectronics,Master’s Degree,"London, Greater London, United Kingdom",NaN,NaN,https://www.linkedin.com/in/apadkavyrava/
761,Konstantinos Bozas,Senior Data Scientist at Amazon | Machine Learning,Amazon,"Queen Mary, U. of London",PhD,"London, United Kingdom",A goal oriented data scientist with research experience in machine learning and computer vision....,NaN,https://www.linkedin.com/in/kbozas/
762,Nadbor Drozd,Lead Data Scientist at Lloyds Banking Group,Lloyds Banking Group,University of Warsaw,M.S,"London, Greater London, United Kingdom",I love solving puzzles. I don't like to babble.\r\n\r\nIf you're a recruiter: http://nadbordrozd...,NaN,https://www.linkedin.com/in/nadbor-drozd-12316063/
763,Christopher Ottesen,Data Scientist,BP,NaN,NaN,"London, Greater London, United Kingdom",NaN,NaN,https://www.linkedin.com/in/christopher-ottesen-4bb10b51/


In [13]:
# find number of missing data based on each column
missing_data = df.isnull()
for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print("\n")

name
False    765
Name: name, dtype: int64


job_title
False    765
Name: job_title, dtype: int64


current_company
False    764
True       1
Name: current_company, dtype: int64


university
False    518
True     247
Name: university, dtype: int64


degree
False    614
True     151
Name: degree, dtype: int64


location
False    765
Name: location, dtype: int64


about
True     422
False    343
Name: about, dtype: int64


skills
True     723
False     42
Name: skills, dtype: int64


present_url
False    765
Name: present_url, dtype: int64




### Do some cleaning and exploring on data
as you can see we have 775 rows × 11 columns. There some rows with null values. Because when we use selenium and getting elements with their class and atributes it has a chance that you got nothing!
We want to explore job applications description so let 's filter data by description

In [ ]:
jobs=df[df['job_description'].notnull()]
jobs

In [ ]:
jobs.columns

In [ ]:
# let 's drop skills and location columns
# because all the rows in skills are null and we scrape United Kingdom so we don't need location anymore!
del jobs['skill']
del jobs['location']

In [ ]:
jobs.dtypes

In [ ]:
jobs.head()

In [ ]:
#let 's see which company has the most applications for data science in united kingdom!
jobs['company'].value_counts().to_frame().reset_index()

In [ ]:
#let 's see what are the most title used for data science jobs in united kingdom!
print(len(jobs['job_title'].unique()) , ' Unique Title')
jobs['job_title'].value_counts().to_frame().reset_index()


In [ ]:
plt.figure(figsize=(10,5))
plt.tight_layout()

jobs['job_title'].value_counts().nlargest(15).plot(kind='bar')


In [ ]:
employement = jobs[jobs['employement_type '].notnull()]

employement['employement_type '].value_counts().to_frame()


In [ ]:
plt.figure(figsize=(10,5))
plt.tight_layout()
employement['employement_type '].value_counts().nlargest(15).plot(kind='bar')


In [ ]:
experience_Level = employement = jobs[jobs['experience_Level'].notnull()]

employement['experience_Level'].value_counts().to_frame()

In [ ]:
plt.figure(figsize=(10,5))
plt.tight_layout()
employement['experience_Level'].value_counts().nlargest(15).plot(kind='bar')


In [ ]:
discription_df = jobs[['job_description']].copy()
discription_df

In [ ]:
discription_df.isna().sum()

In [ ]:
rand_indexs = np.random.randint(1,len(discription_df),50).tolist()
discription_df["job_description"][rand_indexs]

In [ ]:
import string
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    text = re.sub(r'[^a-zA-Z0-9 \n\.]', ' ' , text)
    text = text.replace('\n', ' ')
    return text

discription_df['description_punct'] = discription_df["job_description"].apply(lambda x: remove_punct(x))
discription_df.head(10)

In [ ]:
def tokenization(text):
    text = re.split('\W+', text)
    return text

discription_df['discription_tokenized'] = discription_df['description_punct'].apply(lambda x: tokenization(x.lower()))
discription_df.head()

In [ ]:
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text
    
discription_df['discription_nonstop'] = discription_df['discription_tokenized'].apply(lambda x: remove_stopwords(x))
discription_df.head(10)

In [ ]:
ps = nltk.PorterStemmer()

def stemming(text):
    text = [ps.stem(word) for word in text]
    return text

discription_df['discription_stemmed'] = discription_df['discription_nonstop'].apply(lambda x: stemming(x))
discription_df.head()

In [ ]:
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    text = [wn.lemmatize(word) for word in text]
    return text

discription_df['discription_lemmatized'] = discription_df['discription_nonstop'].apply(lambda x: lemmatizer(x))
discription_df.head()

In [ ]:
from nltk.tokenize import word_tokenize

# Uncomment this line if you haven't downloaded punkt before
# or just run it as it is and uncomment it if you got an error.
# nltk.download('punkt')

def most_used_words(text):
    tokens = word_tokenize(text)
    frequency_dist = nltk.FreqDist(tokens)
    print("There is %d different words" % len(set(text)))
    return sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)

In [ ]:
words = set(nltk.corpus.words.words())

test_text_data = " ".join(w for w in nltk.wordpunct_tokenize(discription_df.job_description.str.cat()) \
         if w.lower() in words or not w.isalpha())


In [ ]:
most_used_words(test_text_data)[:10]

In [ ]:
mw = most_used_words(discription_df.job_description.str.cat())
most_words = []
for w in mw:
    if len(most_words) == 1000:
        break
    if w in stopwords.words("english"):
        continue
    else:
        most_words.append(w)

In [ ]:
sorted(most_words)

In [ ]:
# I'm defining this function to use it in the 
# Data Preparation Phase
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

# nltk.download('wordnet')
def stem_tokenize(text):
    stemmer = SnowballStemmer("english")
    stemmer = WordNetLemmatizer()
    return [stemmer.lemmatize(token) for token in word_tokenize(text)]

def lemmatize_tokenize(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in word_tokenize(text)]

In [ ]:
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import matplotlib.pyplot as plt
# Start with one review:
text = " ".join(discription for discription in discription_df.description_punct)
# lower max_font_size, change the maximum number of word and lighten the background:
stopwords = set(STOPWORDS)
# stopwords.add(search_modifier)
wordcloud = WordCloud(max_font_size=50, max_words=100,stopwords= stopwords ,  background_color="white").generate(text)
wordcloud.to_file('N.png')
plt.figure(figsize=(100,50))
plt.tight_layout()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()
# plt.savefig('wordcloud.png')

## Frequency Analysis

In [ ]:
#download nltk attribute if needed
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('reuters')

In [ ]:
text = ' '.join(description for description in discription_df.description_punct)

In [ ]:
text

In [ ]:
# tokenization
filtered_text = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]

In [ ]:
from nltk.corpus import stopwords
# stopwords
filtered_text = [word for word in filtered_text if word not in stopwords.words('english')]

In [ ]:
# punctuation
filtered_text = [''.join(c for c in s if c not in string.punctuation) for s in filtered_text]
filtered_text = [s for s in filtered_text if s]

In [ ]:
# lowercase
filtered_text = [word.lower() for word in filtered_text]

In [ ]:
# Word Stemming
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
filtered_text_stem = [stemmer.stem(t) for t in filtered_text]

In [ ]:
# Lemmatization
lemmatizer = nltk.WordNetLemmatizer()

filtered_text_lem = [lemmatizer.lemmatize(t) for t in filtered_text]

In [ ]:
# NLTK Frequency Distributions
#nltkText = [filtered_text_stem, filtered_text_lem]

fdist = nltk.FreqDist(filtered_text_lem)

fdistDF = pd.DataFrame.from_dict(fdist, orient='index').reset_index()
fdistDF = fdistDF.rename(columns={'index':'word', 0:'count'})
fdistDF = fdistDF.sort_values(by='count', ascending=False)
print('-----------------head words-----------------')
print(fdistDF.head(10).reset_index(drop=True))
print('-----------------tail words-----------------')
print(fdistDF.tail(10).reset_index(drop=True))

## Analyze document words with data science job descripiton database
To indentify what skills are specific for this job vacancy in particulair we have to create a data science job description database. Using this database we can identify what is specific about this job with the steps below.

Building a Term-Document Matrix
Use scikit-learn's TfidfVectorizer class to construct a term-document matrix containing the TF-IDF score for each word in each document in the data science job description database. In essence, the rows of this sparse matrix correspond to documents in the corpus, the columns represent each word in the vocabulary of the corpus, and each cell contains the TF-IDF value for a given word in a given document.

TF-IDF Scores
Now that we've built the term-document matrix, we can explore its contents:

In [ ]:
from random import randint
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


# rand_indexs = np.random.randint(1,len(discription_df),1)
rand_indexs = np.random.randint(1,len(discription_df),1)
text = " ".join(discription_df["description_punct"][rand_indexs].tolist())
print(text)

tfidf = TfidfVectorizer(stop_words='english', analyzer='word', lowercase=True)
content = text.split()
tfidf_vector = tfidf.fit_transform(content)

feature_names = tfidf.get_feature_names()

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(content)

tdm= pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
print ('TDM contains ' + str(len(feature_names)) + ' terms and ' + str(tdm.shape[0]) + ' documents')

print ('first term: ' + feature_names[0])
print ('last term: ' + feature_names[len(feature_names) - 1])

for i in range(0, 4):
    print ('random term: ' + feature_names[randint(1,len(feature_names) - 2)])

In [ ]:
discription_df.columns

### Generating the Summary
That's all we'll need to produce a summary for any document in the corpus. In the example code below, we start by randomly selecting an article from the data science job description database. We iterate through the article, calculating a score for each sentence by summing the TF-IDF values for each word appearing in the sentence. We normalize the sentence scores by dividing by the number of tokens in the sentence (to avoid bias in favor of longer sentences). Then we sort the sentences by their scores, and return the highest-scoring sentences as our summary. The number of sentences returned corresponds to roughly 20% of the overall length of the article.

In [ ]:
from __future__ import division
import math

def tokenize_and_stem(sentence):
    filtered_text = [word for sent in nltk.sent_tokenize(sentence) for word in nltk.word_tokenize(sent)]
    stemmer = SnowballStemmer("english")
    filtered_text_stem = [stemmer.stem(t) for t in filtered_text]
    return filtered_text_stem
    
    
    
article_id = randint(0, tdm.shape[0] - 1)
article_text = discription_df.iloc[article_id]
print(article_text)

sent_scores = []
for sentence in nltk.sent_tokenize(article_text):
    score = 0
    sent_tokens = tokenize_and_stem(sentence)
    for token in (t for t in sent_tokens if t in feature_names):
        score += tdm[article_id, feature_names.index(token)]
    sent_scores.append((score / len(sent_tokens), sentence))

summary_length = int(math.ceil(len(sent_scores) / 5))
sent_scores.sort(key=lambda sent: sent[0], reverse=True)

print ('*** SUMMARY ***')
for summary_sentence in sent_scores[:summary_length]:
    print (summary_sentence[1])

print ('\n*** ORIGINAL ***')
print (article_text)